In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# Training settings
batch_size = 64

In [3]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

100%|██████████| 9912422/9912422 [00:01<00:00, 9540135.75it/s] 


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 5967275.56it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2182538.09it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 8286441.40it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



`nn.Conv2d(in_channels, out_channels, kernel_size)`

- `in_channels`: 입력 채널의 개수, RGB 이미지의 경우 3개 채널이라 in_channels = 3
- `out_channels`: 출력 채널의 개수, 컨볼루션 필터의 개수이다. 컨볼루션 이후 생성되는 feature map의 개수를 조정한다. 이 값이 클수록 더 복잡한 특징을 학습할 수 있지만 모델의 파라미터 수가 증가한다.
- `kernel_size`: 컨볼루션 필터의 크기(size), 3x3 필터의 경우 kernel_size = 3이다.

Reference: https://wikidocs.net/194946

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20,  kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)
        
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1) # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [6]:
model = Net()

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
def train(epoch):
    model.train()  # 모델을 학습 모드로 설정
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target) # nll_loss를 쓰기 때문에 forward 마지막에 log_softmax가 존재함. 
        # CrossEntropyLoss = log_softmax + nll_loss
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
def test():
    model.eval() # 모델을 평가 모드로 설정. 모델이 추론 시에 학습 모드와 동일한 동작을 수행함으로써 불필요한 노이즈 줄이고 일관된 결과 획득 가능
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
for epoch in range(1, 10):
    train(epoch)
    test()

/var/folders/n_/nbl4pgg93n143h_gp4rbmncr0000gn/T/ipykernel_38512/4203862180.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309096
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.287458
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.265192
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.259687
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.248495
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.222260
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.151794
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.075187
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.012360
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.911282
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.823573
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.575044
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.336385
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.249550
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.890443
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.036583
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.767496
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.712877
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.886275
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.430604
Train Epoch: 1 [12800/60000 (

/var/folders/n_/nbl4pgg93n143h_gp4rbmncr0000gn/T/ipykernel_38512/3167063005.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
/Users/jaewoogwak/anaconda3/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1697, Accuracy: 9526/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.069462
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.257750
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.156426
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.335267
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.382693
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.084234
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.169049
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.115809
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.079556
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.086690
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.109768
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.317690
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.415950
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.233863
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.156266
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.118346
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.184006
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.273345
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.259191
Train Epoch: 2 [12